In [ ]:
import numpy as np

In [ ]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm, skew 
from scipy import stats 
from matplotlib import rcParams


In [ ]:
import statsmodels.api as sm

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
df = pd.read_csv('/home/bami123/Historical Product Demand.csv')

In [ ]:
df.head()

In [ ]:

print(df.shape)
print(df.dtypes)

In [ ]:
df.isnull().sum()
# calculating % of data missing
print("% of Data missing =", df.isnull().sum().sum()/len(df)*100)


In [ ]:

df.dropna(axis=0, inplace=True) 

#setting date as index columns
df.reset_index(drop = True)
df.isnull().sum()


In [ ]:
df.sort_values('Date')[10:20]

In [ ]:
df['Order_Demand'] = df['Order_Demand'].str.replace('(',"")
df['Order_Demand'] = df['Order_Demand'].str.replace(')',"")
df.sort_values('Date')[10:20]

In [ ]:
df['Order_Demand'] = df['Order_Demand'].astype('int64')

In [ ]:
sns.set(rc={'figure.figsize':(16,5)})
sns.distplot(df['Order_Demand'], bins = 100);

In [ ]:
rcParams['figure.figsize'] = 50,14
sns.barplot(x=pd.DatetimeIndex(df['Date']).year, y=df['Order_Demand'])

In [ ]:
import seaborn as sns
rcParams['figure.figsize'] = 5,3
sns.distplot(df['Order_Demand'], fit=norm)

fig = plt.figure()
res = stats.probplot(df['Order_Demand'], plot=plt)
plt.show()

In [ ]:
sns.lineplot(x=df['Date'], y=df['Order_Demand'])


In [ ]:
df.groupby('Warehouse')['Order_Demand'].sum().sort_values(ascending=False)

In [ ]:
df1 = pd.DataFrame(df.groupby('Product_Category')['Order_Demand'].sum().sort_values(ascending=False))
df1["% Contribution"] = df1['Order_Demand']/df1['Order_Demand'].sum()*100
df1

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df2 = pd.pivot_table(df,index=["Date"],values=["Order_Demand"],columns=["Product_Category"],aggfunc=np.sum)
df2.columns = df2.columns.droplevel(0)
df2["Category_019"].dropna()

In [ ]:
y = df2.resample('M').sum() # Resampling the data on monthly basis 
y.index.freq = "M" # Setting datetime frequency to Month
y.head(20)

In [ ]:
df_019 = pd.DataFrame(y["Category_019"].iloc[12:-1]) 
df_019.head()

In [ ]:
predictors=['Product_Code','Warehouse','Product_Category','Date']
X=df[predictors]
y=df.Order_Demand
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1,test_size=0.1,train_size=0.5)
one_hot_encoding_val_data=pd.get_dummies(val_X)
one_hot_encoding_train_data=pd.get_dummies(train_X)
del val_X
del train_X
final_train, final_val = one_hot_encoding_train_data.align(one_hot_encoding_val_data,join='left',axis=1)

del one_hot_encoding_val_data
del one_hot_encoding_train_data

final_train = Imputer.fit_transform(final_train)
final_val = Imputer.transform(final_val)
#defining the model 
my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(final_train, train_y, verbose=False)
del final_train
del train_y

predictions = my_model.predict(final_val)
del final_val
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, val_y)))
